## Explore And Clean `DataWorkOutRun.csv`

**Purpose :** of this notebook is to clean DataWorkoutRun.csv in order to aggregate it (Part B. Notebook2) and then add subscription column. Following that I will build machine learning model to undertsand what user behavior drives their subscription DNA (what knowledge we can drive from their behavior to understand whether a user will subscribe).

In [1]:
import numpy as np
import pandas as pd
# ignore warnings
import warnings
warnings.filterwarnings('ignore')

### Checking and labelling the unique_id of the ongouser_modeling data #####

In [2]:
df_x = pd.read_csv('ongouser_modeling_ds-2.csv', parse_dates=['joined_community_at','converted_to_started_subscription_at'])
df_x.head(2)

,Unnamed: 0,joined_community_at,community_type,goal,community,metric_started_app_session_week1,metric_started_app_session_week2,metric_started_app_session_week3,metric_started_app_session_week4,metric_complete_session_gettingstarted_week4,...,metric_session_start_day_0,metric_session_start_day_1_30,metric_session_start_day_30_60,metric_session_start_day_60_up,_id,flag_startsession_b4_subs,flag_starttrial_b4_subs,flag_companysession_b4_subs,flag_compquickstart_b4_subs,flag_enrolprgm_b4_subs
0,1,2020-01-17 15:50:00,running,running.health,The Run Experience,1.0,1.0,1.0,1.0,1.0,...,1,0,0,0,bc03395a-222e-42d3-be3f-d88878b57640,NaN,NaN,NaN,NaN,NaN
1,2,2020-01-17 15:43:00,running,running.race,The Run Experience,1.0,1.0,1.0,1.0,0.0,...,0,0,0,0,c4b763d9-2733-4d89-b889-3b2d92e8e2a1,NaN,NaN,NaN,NaN,NaN


In [3]:
print(f'Shape of Ongouser_modeling_ds rows: {df_x.shape[0]}, columns: {df_x.shape[1]}')

# Label the unique ids of ongouser with easy to read ids
dicti = {}
x=0
for i in df_x['_id'].unique():  
    dicti[i]=x
    x+=1
df_x['easy_id_4_subscription']=df_x['_id'].map(dicti)

dicti2 = {}
for index, row in df_x.iterrows():
    dicti2[row['easy_id_4_subscription']]=row['joined_community_at']

Shape of Ongouser_modeling_ds rows: 19778, columns: 45


In [4]:
# Create a new date mapped specific to each user (2 months after joining the community)
# 2 months because we are trying to predict subscription 2 months after joining the community (Free-tier)
df_x['2month_4m_joining'] = df_x.apply(lambda x: x['joined_community_at'] + pd.DateOffset(months = 2), axis=1)

#Unique ongousers in Ongouser_modeling_ds 
df_x['easy_id_4_subscription'].nunique()

19778

#### Mapping the user_ids of DataWorkoutRun ( dataframe that needs to be aggregated) with same ids from other .csv###

In [5]:
df  = pd.read_csv('DataWorkoutRun_with_header.csv')
# Location info not required
df = df.drop(columns=['locations'])

# Map the owner ids with the easy_ids from Ongouser_modeling_ds 
df['easy_id_2be_aggregated']=df['owner'].map(dicti)
df[['owner','easy_id_2be_aggregated']].nunique()

owner                     1097
easy_id_2be_aggregated    1030
dtype: int64

Some owners (67) in DataWorkoutRun are not present in the Ongo-modeling data, so I have to remove them

In [6]:
df_y = df.dropna(subset=['easy_id_2be_aggregated'])
df_y[['owner','easy_id_2be_aggregated']].nunique()

owner                     1030
easy_id_2be_aggregated    1030
dtype: int64

In [7]:
print(f'Shape of DataWorkoutRun rows: {df_y.shape[0]}, columns: {df_y.shape[1]}')

Shape of DataWorkoutRun rows: 5591, columns: 15


In [8]:
# Startdate needs to be datetime column
df_y['startDate'] = pd.to_datetime(df_y['startDate'], errors='coerce')
# Timezone info has to be removed to compare datetime columns
df_y['startDate'] = df_y['startDate'].dt.tz_convert(None)

# finding common ids between Ongouser_modeling and DataWorkOutRun
common_id_finding = df_y['easy_id_2be_aggregated'].to_list()
filt = df_x['easy_id_4_subscription'].isin(common_id_finding)
print(f'Before selecting only common users, Ongouser_modeling data had rows: {df_x.shape[0]} and columns: {df_x.shape[1]}')
common_agg_df = df_x[filt]
print(f'After selecting only common users, Ongouser_modeling data had rows: {common_agg_df.shape[0]} and columns: {common_agg_df.shape[1]}')

Before selecting only common users, Ongouser_modeling data had rows: 19778 and columns: 47
After selecting only common users, Ongouser_modeling data had rows: 1030 and columns: 47


Users who has subscribed (my main target in the Ongouser_modeling data), we know their date of joining Ongo_community (`joined_community_at`) and their date of subscribing (`converted_to_started_subscription_at`). We need to see the activity of the subscribers (between these two dates) and Non-subscribers (for whatever timeframe data available for them {maximum 2 months is considered} as they never subscribed). So for that I need to filter out any data I have for subscribers beyond the subscription date.

In [9]:
# Only considering the users that subscribed
common_agg_df2 = common_agg_df[common_agg_df['converted_to_started_subscription_at'].notna()]
print(f'Total people who subscribed out of {common_agg_df.shape[0]} Ongo users : {common_agg_df2.shape[0]}')

Total people who subscribed out of 1030 Ongo users : 147


Selecting the Subscribed users from DataWorkOutRun (only those rows that are between `joined_community_at` and `converted_to_started_ subscription_at` dates)

In [10]:
from tqdm import tqdm_notebook

agg = [] 
for index1,row1 in tqdm_notebook(df_y.iterrows()):
    for index2,row2 in common_agg_df2.iterrows():
        if row1['easy_id_2be_aggregated'] == row2['easy_id_4_subscription']:
            if (row1['startDate']<=row2['converted_to_started_subscription_at']):
                if (row1['startDate']<=row2['2month_4m_joining']):
                    agg.append(row1)
                    
# Subscribed user info from DataWorkoutRun between joined_community_at & converted_to_started_ subscription_at dates
agg1_df = pd.DataFrame(agg)
agg1_df.shape

(198, 15)

In [11]:
# Only considering the users that DID NOT subscribe
common_agg_df3 = common_agg_df[common_agg_df['converted_to_started_subscription_at'].isna()]
print(f'Total people who did NOT subscribed out of {common_agg_df.shape[0]} Ongo users : {common_agg_df3.shape[0]}')

Total people who did NOT subscribed out of 1030 Ongo users : 883


Selecting the NON-Subscribed users from DataWorkOutRun, for them the range of timeframe would be 2 months. 

In [12]:
agg2 = [] 
for index1,row1 in tqdm_notebook(df_y.iterrows()):
    for index2,row2 in common_agg_df3.iterrows():
        if row1['easy_id_2be_aggregated'] == row2['easy_id_4_subscription']:
            if (row1['startDate']<=row2['2month_4m_joining']):
                agg2.append(row1)
                
# NON-Subscribed user info from DataWorkoutRun 
agg2_df = pd.DataFrame(agg2)
agg2_df.shape

(2578, 15)

Joining the rows from DataWorkoutRun for subscribers (between `joined_community_at` and `converted_to_started_ subscription_at` dates) and Non-subscribers

In [13]:
final_df_2be_agg = agg1_df.append(agg2_df, ignore_index=True)
# loss of some subscribed user info because of errors in the data (datetime column)
final_df_2be_agg['easy_id_2be_aggregated'].nunique()

821

In [14]:
final_df_2be_agg['joined_community_at'] = final_df_2be_agg['easy_id_2be_aggregated'].map(dicti2)
print(f'Overall shape of data that I need to aggregate, rows: {final_df_2be_agg.shape[0]} and columns: {final_df_2be_agg.shape[1]}')

final_df_2be_agg.to_csv("clean_DataWorkOutRun.csv", index=True)
df = pd.read_csv('clean_DataWorkOutRun.csv')
df.head(2)

Overall shape of data that I need to aggregate, rows: 2776 and columns: 16


,Unnamed: 0,type,owner,session,sessionElement,track,sessionTitle,trackTitle,startDate,startLocation,runDuration,runDistance,createdAt,updatedAt,timezone,easy_id_2be_aggregated,joined_community_at
0,0,og.outdoor,2bb53a3f-a458-4c0d-8339-b30492b1e5f6,1d5453c4-a535-45df-9047-65d2eb00ad9f,33b57178-3213-4596-954f-96c4db9ad2a7,bc94350f-9201-4bc0-b792-f5f4abdcf6a0,Day 9: Drill Day,Marathon Program,2019-08-13 11:07:40.168,0101000020E610000018F548B4F677434017C29DDC933D...,892.070895,2351.482732,2019-08-13 11:37:42.775+00,2019-08-13 11:37:42.775+00,America/New_York,19108.0,2019-06-21 07:49:00
1,1,og.outdoor,4dc4c6f6-2ce5-4620-ba5c-2ed4b869998d,bbabe385-8cbd-41ac-b1e1-fea8cbef1973,21cf4c3e-e1e3-406b-a90e-4828263f038e,544e3df1-02f9-4086-8f53-7922257684c5,Day 2: Drill Day,30 Day Challenge,2019-08-13 01:02:51.095,0101000020E610000091428B2F3C44454025859E544CCC...,627.019304,1694.252356,2019-08-13 01:30:21.863+00,2019-08-13 01:30:21.863+00,America/Detroit,19074.0,2019-06-22 16:14:00


**Next step: Aggregate this df, add binary (whether user synced their historic data from 3rd party integration) and other important features that separate the clusters to the aggregated data and then predict subscription**

In [ ]:
#### Next: Part B. 2. Aggregating_DataWorkoutRun_with_AppleHealthKit_G.ipynb ####